In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install ultralytics
!pip install opencv-python

# Import necessary libraries
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 49.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Pose Estimation

In [ ]:
# model
yolo_model = YOLO("yolo11n-pose.pt")

# Activity
activity_videos = {
    'sitting': ['/content/drive/MyDrive/P2/DATA/Sitting/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Sitting/002.mp4',
                #'/content/drive/MyDrive/P2/DATA/Sitting/003.mp4',
                #'/content/drive/MyDrive/P2/DATA/Sitting/004.mp4',
                #'/content/drive/MyDrive/P2/DATA/Sitting/005.mp4'
                ],
    'laying': ['/content/drive/MyDrive/P2/DATA/Laying/001.mp4',
               '/content/drive/MyDrive/P2/DATA/Laying/002.mp4',
               #'/content/drive/MyDrive/P2/DATA/Laying/003.mp4',
               #'/content/drive/MyDrive/P2/DATA/Laying/004.mp4',
               #'/content/drive/MyDrive/P2/DATA/Laying/005.mp4'
               ],
    'jumping': ['/content/drive/MyDrive/P2/DATA/Jumping/001.mp4',
                '/content/drive/MyDrive/P2/DATA/Jumping/002.mp4',
                #'/content/drive/MyDrive/P2/DATA/Jumping/003.mp4',
                #'/content/drive/MyDrive/P2/DATA/Jumping/004.mp4',
                #'/content/drive/MyDrive/P2/DATA/Jumping/005.mp4'
                ]
}

100%|██████████| 5.97M/5.97M [00:00<00:00, 378MB/s]


In [ ]:
# List to store keypoints data for all activities
all_activity_points = []

# Loop
for activity, video_paths in activity_videos.items():
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)

        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = cap.get(cv2.CAP_PROP_FPS)
        seconds = round(frames / fps)

        frame_total = 30  # Number of frames to process
        i, a = 0, 0

        while cap.isOpened():
            cap.set(cv2.CAP_PROP_POS_MSEC, (i * ((seconds / frame_total) * 1000)))
            flag, frame = cap.read()

            if not flag:
                break

            # Perform YOLO pose estimation on the frame
            results = yolo_model(frame, verbose=False)

            for r in results:
                bound_box = r.boxes.xyxy
                conf = r.boxes.conf.tolist()
                keypoints = r.keypoints.xyn.tolist()

                for index, box in enumerate(bound_box):
                    if conf[index] > 0.75:
                        x1, y1, x2, y2 = box.tolist()
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                        label = f'{activity}'
                        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                        skeleton_connections = [(0, 1), (1, 2), (2, 3), (3, 4),  # Left arm
                                                (0, 5), (5, 6), (6, 7), (7, 8),  # Right arm
                                                (9, 10), (10, 11), (11, 12),     # Left leg
                                                (9, 13), (13, 14), (14, 15),     # Right leg
                                                (0, 9)]                          # Spine
                        for p1, p2 in skeleton_connections:
                            x1, y1 = keypoints[index][p1]
                            x2, y2 = keypoints[index][p2]
                            cv2.line(frame, (int(x1 * frame.shape[1]), int(y1 * frame.shape[0])),
                                     (int(x2 * frame.shape[1]), int(y2 * frame.shape[0])), (0, 0, 255), 2)

                        output_path = f'/content/drive/MyDrive/P2/YOLO/output/{activity}_out_{a}_skeleton.jpg'
                        cv2.imwrite(output_path, frame)

                        data = {'image_name': f'{activity}_person_{a}.jpg', 'activity': activity}
                        for j in range(len(keypoints[index])):
                            data[f'x{j}'] = keypoints[index][j][0]
                            data[f'y{j}'] = keypoints[index][j][1]

                        all_activity_points.append(data)
                        a += 1
            i += 1

        cap.release()

# Save keypoints data into a CSV
df = pd.DataFrame(all_activity_points)
csv_file_path = '/content/drive/MyDrive/P2/DATA/activity_keypoints_01.csv'
df.to_csv(csv_file_path, index=False)
print(f"Keypoints data saved to {csv_file_path}")

Keypoints data saved to /content/drive/MyDrive/P2/DATA/activity_keypoints_01.csv


Dataset

In [ ]:
# Training
### Change this part!!
df = pd.read_csv(csv_file_path)
X = df.drop(['image_name', 'activity'], axis=1)
y = df['activity'].astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

Activity Classification

In [ ]:
# neural network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))

# Modle Compile and train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_data=(X_test, y_test_cat))

# Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.2777 - loss: 1.1597 - val_accuracy: 0.0962 - val_loss: 1.1168
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1308 - loss: 1.1091 - val_accuracy: 0.1538 - val_loss: 1.0976
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2821 - loss: 1.0782 - val_accuracy: 0.2308 - val_loss: 1.0807
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4538 - loss: 1.0523 - val_accuracy: 0.5385 - val_loss: 1.0654
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6840 - loss: 1.0166 - val_accuracy: 0.5577 - val_loss: 1.0491
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6971 - loss: 0.9993 - val_accuracy: 0.5769 - val_loss: 1.0279
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6852 - loss: 0.9745 - val_accuracy: 0.6346 - val_loss: 1.0009
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7921 - loss: 0.9661 - val_accuracy: 0.8846 - val_loss: 0.9793
Epoch 9/20
4/

web cam

In [ ]:
# Real-time activity recognition with webcam
yolo_model = YOLO("yolo11n-pose.pt")
classifier_model = model

cap = cv2.VideoCapture(0)
output_video_path = '/content/drive/MyDrive/P2/YOLO/output/activity_recognition_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

keypoints_data = []
activity_labels = {0: 'sitting', 1: 'laying', 2: 'jumping'}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame, verbose=False)

    keypoints_list = []
    for r in results:
        keypoints = r.keypoints.xyn.tolist()
        for keypoints_per_person in keypoints:
            keypoints_flat = [coord for kp in keypoints_per_person for coord in kp]
            keypoints_list.append(keypoints_flat)

    if keypoints_list:
        keypoints_array = np.array(keypoints_list)
        predictions = classifier_model.predict(keypoints_array)
        predicted_activity = np.argmax(predictions, axis=1)

        for i, activity in enumerate(predicted_activity):
            label = activity_labels[activity]
            print(f"Person {i+1} activity: {label}")

            keypoints = keypoints_list[i]
            x1, y1 = int(keypoints[0] * frame_width), int(keypoints[1] * frame_height)
            x2, y2 = int(keypoints[6] * frame_width), int(keypoints[7] * frame_height)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            data = {'frame': len(keypoints_data) + 1, 'activity': label}
            for j in range(0, len(keypoints), 2):
                data[f'x{j//2}'] = keypoints[j]
                data[f'y{j//2}'] = keypoints[j + 1]
            keypoints_data.append(data)

    out.write(frame)
    cv2.imshow('Activity Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Save real-time keypoints data to CSV
df_keypoints = pd.DataFrame(keypoints_data)
keypoints_csv_path = '/content/drive/MyDrive/P2/DATA/webcam_keypoints.csv'
df_keypoints.to_csv(keypoints_csv_path, index=False)

print(f"Video saved to {output_video_path}")
print(f"Keypoints data saved to {keypoints_csv_path}")

Video saved to /content/drive/MyDrive/P2/YOLO/output/activity_recognition_output.mp4
Keypoints data saved to /content/drive/MyDrive/P2/DATA/webcam_keypoints.csv
